In [29]:
import os
import random
import pandas as pd
import numpy as np
from datetime import datetime
import time
#from Levenshtein import distance
from scipy.spatial.distance import pdist, squareform
from scipy import sparse, io

In [30]:
def homogenized(strings):
    lengths = [len(s) for s in strings]
    n = max(lengths)
    for s in strings:
        k = len(s)
        yield [k] + [ord(c) for c in s] + [0] * (n - k)

In [2]:
os.chdir("../output/")

In [10]:
random.seed(9999)
print(random.random())
#This checks the current working directory
# print("Current Working Directory " , os.getcwd())

0.8347577610922152


In [24]:
#The goal is to match row data with distance matrix
sample_id = "TRB-Pt-10-2"
print(sample_id)
parent_dir = "./"
directory = sample_id + "_hamming/"
path = os.path.join(parent_dir, directory)
#Equiivalent of the dir.create function in R
#os.mkdir(path)
input_data = pd.read_table("../data/TRB-Pt-10-2-500ng-15-04-2020-gDNA_S48.clones.txt", delimiter = '\t')

TRB-Pt-10-2


In [25]:
print(len(np.unique(input_data['aaSeqCDR3'])))
nSeqCDR3list = list(input_data['nSeqCDR3'])
aaSeqCDR3list = list(input_data['aaSeqCDR3'])
cloneCountlist = list(input_data['cloneCount'])
cloneFractionlist  = list(input_data['cloneFraction'])

sub_input = pd.DataFrame(dict(nSeqCDR3list = nSeqCDR3list, aaSeqCDR3list = aaSeqCDR3list, cloneCountlist = cloneCountlist, cloneFractionlist = cloneFractionlist))
sub_input2 = pd.DataFrame(dict(aaSeqCDR3list =  aaSeqCDR3list, aaSeqCDR3list2 = aaSeqCDR3list, cloneCountlist = cloneCountlist, cloneFractionlist = cloneFractionlist)) 


2906


In [26]:
agg_count_on_aa = sub_input2[['cloneCountlist', 'cloneFractionlist', 'aaSeqCDR3list2']].groupby(aaSeqCDR3list).agg(['sum'])
agg_count_on_aa.columns = ['cloneCount', 'cloneFraction', 'aaSeqCDR3']
values = sub_input['aaSeqCDR3list'].value_counts().keys().tolist()
counts = sub_input['aaSeqCDR3list'].value_counts().tolist()
unique_aa_data = pd.DataFrame({'aaSeqCDR3' : values,
                                'unique_nucleotide_count' : counts,
                                }, 
                                columns=['aaSeqCDR3', 'unique_nucleotide_count'])

print(unique_aa_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906 entries, 0 to 2905
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   aaSeqCDR3                2906 non-null   object
 1   unique_nucleotide_count  2906 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 45.5+ KB
None


In [27]:
sub_input_unique_count = agg_count_on_aa.merge(unique_aa_data, how = 'left', left_on = "aaSeqCDR3", right_on = "aaSeqCDR3")
print(sub_input_unique_count.info())

input_raw_data = sub_input_unique_count
print(input_raw_data.info())
input_raw_data_unique = input_raw_data
input_raw_data_unique.duplicated(subset = "aaSeqCDR3", keep = 'first')
print(input_raw_data_unique.info())
input_raw_data = input_raw_data_unique


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2906 entries, 0 to 2905
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   cloneCount               2906 non-null   float64
 1   cloneFraction            2906 non-null   float64
 2   aaSeqCDR3                2906 non-null   object 
 3   unique_nucleotide_count  2841 non-null   float64
dtypes: float64(3), object(1)
memory usage: 113.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2906 entries, 0 to 2905
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   cloneCount               2906 non-null   float64
 1   cloneFraction            2906 non-null   float64
 2   aaSeqCDR3                2906 non-null   object 
 3   unique_nucleotide_count  2841 non-null   float64
dtypes: float64(3), object(1)
memory usage: 113.5+ KB
None
<class 'pandas.core.fr

In [28]:
start_time = datetime.now()
input_raw_data['AA_length'] = input_raw_data['aaSeqCDR3'].str.len()
    
sub_input_raw = input_raw_data[(input_raw_data['AA_length'] > 2 ) & (input_raw_data['aaSeqCDR3'] != "")]
input_AA = sub_input_raw['aaSeqCDR3'].astype(str)
print(len(input_AA))
if len(input_AA) == 1:
    next()


2906


In [31]:
start_time = time.time()

In [32]:
strings = input_raw_data['aaSeqCDR3'].astype(str)
strings = input_AA.astype(str)
# input_string = strings[0].tolist()
input_string = strings.tolist()
#input_string = x

points = np.array(list(homogenized(input_string)))
d = squareform(pdist(points, metric='hamming'))
second_min = np.unique(d)[1]

d = np.where(d <= second_min, 99 , d)
d = np.where(d <= 1, 0 , d)
d = np.where(d == 99 , 1 , d)
select_col_id = (d.sum(axis = 1) > 1)
d = d[select_col_id]
d = d[:,np.array(select_col_id)]


In [33]:
# write distance matrix into file
io.mmwrite('hamming_distance_martrix_degree_lt0.mtx', sparse.csr_matrix(d))
# np.savetxt("select_col_id.csv", select_col_id, delimiter=",")

In [34]:
print(len(input_string))
print(d.shape)
print('--- %s seconds ---' % (time.time() - start_time))

#rename the hamming distance file by sample_id and Vgene
os.rename('hamming_distance_martrix_degree_lt0.mtx', 'distance_matrix_betaCDR3_Europe_covid_data_'+ sample_id + '.mtx')


2906
(403, 403)
--- 21.666754007339478 seconds ---
